In [44]:
import warnings

warnings.filterwarnings("ignore")

from common_utils.utils.config import Config
from common_utils.io.data_access.data_access_factory import DataAccessFactory

# from axpo_trading.forecast.forecast_preprocess_iberia import preproces_ufis
from common_utils.utils import utils, utils_io, utils_date
from axpo_trading.forecast import forecast_sql_preprocess_iberia
from axpo_trading.forecast import forecast_preprocess_iberia
import pandas as pd
import numpy as np
import os
import datetime
from numpy import array
import matplotlib.pyplot as plt

# Random seeds
from numpy.random import seed

seed(42)
from tensorflow.keras.utils import set_random_seed

set_random_seed(42)
import random as rn

rn.seed(1254)
from keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# wind_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
wind_path = "/home/jovyan/projects/AdvancedAnalytics-UseCase-Wind"
os.chdir(wind_path)

os.environ["CONFIG_DIR"] = "config_files"
os.environ["AUTH_CONFIG_DIR"] = "auth"
os.environ["AZURE_STORAGE_ACCOUNT_RAW_CONTAINER_NAME_WIND_RAW"] = "raw"
os.environ["AZURE_STORAGE_ACCOUNT_RAW_CONTAINER_NAME_WIND_STAGING"] = "staging"
os.environ[
    "AZURE_SQL_SHARED_RAW_SERVER"
] = "axso-prod-appl-aa-prod-shared-sql-secondary.database.windows.net"
# os.environ["AZURE_SQL_SHARED_RAW_SERVER"] = 'axso-prod-appl-aa-prod-shared-sql.database.windows.net'
os.environ["AZURE_SQL_SHARED_RAW_DATABASE"] = "axso-prod-appl-aa-prod-shared-raw-sqldb"
os.environ["N_THREADS_SQL"] = "1"

# DEV
os.environ["ENV"] = "azure_iberia_k8s_dev"
# BLOB DEV
os.environ["AZURE_STORAGE_ACCOUNT_DATA_NAME"] = "axsonpaadevdslabdls"
os.environ["AZURE_STORAGE_ACCOUNT_RAW_NAME"] = "axsoprodaaprodshareddls-secondary"
os.environ["AZURE_STORAGE_ACCOUNT_DATA_CONTAINER_NAME_WIND_REFINED"] = "wind-refined"
os.environ["AZURE_STORAGE_ACCOUNT_DATA_CONTAINER_NAME_WIND_RESULTS"] = "wind-results"

In [2]:
n_steps_in = 12
n_steps_out = 3

portfolio_level = True

main_premaster_columns = ["datetime_market","datetime","hours_fwd","ufi","telemetry","forecast","metering"]
info_columns = ["telemetry","forecast","metering","forecast_error_metering","forecast_error_telemetry"]
groupping_columns = ['ufi','hours_fwd']
target_hours_fwd = [1,2,3]
# target_ufis = ["ZAPATER","PEARBO","ROMERA"]
# target_ufis = ["ABELLA","PAXAMON","SPADRON","PELALIN","TIGUEIR","PEIRIXO","MONTCEO","MONTOUT"]
target_ufis= ['ABELLA', 'CERROS', 'LAMESA', 'LACAYA', 'VILACHA', 'TIGUEIR',
       'ESQUILE', 'BRULLES', 'PELALIN', 'PESLOB', 'VISOS', 'DEFERII',
       'PECORTI', 'LASORDA', 'ESCANDO', 'BAYO', 'HINOJII',
       'PEOCHAO', 'CALERA', 'CPELAOS', 'ELGALLO', 'SPADRON', 'PAXAMON',
       'TRAPERA', 'SABUCED', 'PEZARZU', 'PESLOA', 'ASNEVES', 'CAMPANA',
       'PECOUTE', 'HINOJAI', 'PESLOD', 'AXIABRE', 'FEIXOS', 'OTERO',
       'POTRA', 'ZARZUEL', 'CERCEDA', 'GRAIADE', 'PEOUROL', 'RODERA',
       'MONTOUT', 'ARTEIXO', 'ELLLAN', 'MONTCEO', 'LALOMBA', 'CARRACE',
       'PEIRIXO', 'ATALAYA', 'FRAILA', 'DEHESII', 'MONTERO', 'MONDONE',
       'ROMERA', 'ESE', 'BANDELE', 'SANJOSE', 'SERRETA', 'DEHEII',
       'AEROGEN', 'ZAPATER', 'LARUYA', 'PESLOC', 'PEARBO', 'PELALOM',
       'MUDEFER']

## Functions

#### Load master

In [3]:
def pivot_master_by_levels(multiple_line_df):

    # TODO Parametrize levels

    # Pivot data according to level
    # Level 1: market dependent columns
    index_cols = ["datetime_market"]
    market_level_columns = ["hour_market"]
    reduced_df_lv_1 = multiple_line_df[market_level_columns + index_cols]
    reduced_df_lv_1["day_market"] = multiple_line_df["datetime_market"].dt.day
    reduced_df_lv_1["month_market"] = multiple_line_df["datetime_market"].dt.month
    # Add missing levels to even the final shapes
    reduced_df_lv_1 = reduced_df_lv_1.drop_duplicates()
    reduced_df_lv_1 = reduced_df_lv_1.set_index(index_cols, drop=True)
    even_level_1_arrays = [
        market_level_columns + ["day_market", "month_market"],
        [""],
        [""]
    ]
    reduced_df_lv_1.columns = pd.MultiIndex.from_product(even_level_1_arrays, names=["feature", "ufi", "hours_fwd"])

    # Level 2: ufi dependent columns
    index_cols = index_cols + ["ufi"]
    ufi_level_columns = ["p_max", "p_min", "telemetry", "telemetry_pct_good", "telemetry_open", "telemetry_close", "telemetry_min", "telemetry_max", "telemetry_std", "telemetry_value_count", "telemetry_slope", "lat","lon"] # "codCliente", "up", 
    reduced_df_lv_2 = multiple_line_df[ufi_level_columns + index_cols]
    reduced_df_lv_2 = reduced_df_lv_2.drop_duplicates(subset=["datetime_market","ufi"])
    reduced_df_lv_2 = reduced_df_lv_2.pivot(index=['datetime_market'], columns=['ufi'], values=ufi_level_columns)
    # Add missing level to even the shapes
    even_level_2_arrays = [
        list(reduced_df_lv_2.columns.get_level_values(0)),
        list(reduced_df_lv_2.columns.get_level_values(1)),
        list([""] * reduced_df_lv_2.columns.shape[0])
    ]
    even_level_2_tuples = list(zip(*even_level_2_arrays))
    reduced_df_lv_2.columns = pd.MultiIndex.from_tuples(even_level_2_tuples, names=["feature", "ufi", "hours_fwd"])
    reduced_df_lv_2

    # Level 3: horizon dependent columns
    index_cols = index_cols + ["hours_fwd"]
    horizon_level_columns = ["forecast","metering"] #,"forecast_error_metering" ,"forecast_error_telemetry"]
    reduced_df_lv_3 = multiple_line_df[horizon_level_columns + index_cols]
    reduced_df_lv_3 = reduced_df_lv_3.drop_duplicates()
    reduced_df_lv_3 = reduced_df_lv_3.pivot(index=['datetime_market'], columns=['ufi','hours_fwd'], values=horizon_level_columns)

    pivotted_df = pd.concat([reduced_df_lv_1,  pd.concat([reduced_df_lv_2, reduced_df_lv_3], axis=1)], axis=1)

    return pivotted_df



def add_forecast_error_pivot(pivot_df, error_reference="telemetry"):

    ufis_in_df = pivot_df.columns.get_level_values("ufi").unique()
    # Remove empty ufi used for even levels
    ufis_in_df = [ufi for ufi in ufis_in_df if ufi]
    fcst_error_df = pd.DataFrame()
    fcst_error_df_partial = pd.DataFrame()

    for ufi in ufis_in_df:

        if error_reference == "telemetry":
            # Telemetry aligned with index hour (it comes with 1 hour lag)
            telemetry_market_t = pivot_df[error_reference,ufi].shift(-1)
            # Forecasted production aligned with the index hour (we take the t+1 forecast)
            forecast_market_t = pivot_df["forecast",ufi,1].shift(1)
            fcst_error_df_partial[f"forecast_error_{error_reference}"] = forecast_market_t - telemetry_market_t
            # Lag the forecast error 1 hour so it is available at prediction time
            fcst_error_df_partial[f"forecast_error_{error_reference}"] = fcst_error_df_partial[f"forecast_error_{error_reference}"].shift(1)
        else:
            # Error with respect to Metering  which is already aligned
            metering_market_t = pivot_df[error_reference,ufi,1]
            forecast_market_t = pivot_df["forecast",ufi,1]
            fcst_error_df_partial[f"forecast_error_{error_reference}"] = forecast_market_t - metering_market_t
            # Lag the forecast error 1 hour so it is available at prediction time
            fcst_error_df_partial[f"forecast_error_{error_reference}"] = fcst_error_df_partial[f"forecast_error_{error_reference}"]


        fcst_error_df_partial["ufi"] = ufi
        fcst_error_df = pd.concat([fcst_error_df, fcst_error_df_partial])

    fcst_error_df = fcst_error_df.pivot(columns=['ufi'], values=[f"forecast_error_{error_reference}"])

    # Add missing level to even the shapes
    even_level_2_arrays = [
        list(fcst_error_df.columns.get_level_values(0)),
        list(fcst_error_df.columns.get_level_values(1)),
        list([""] * fcst_error_df.columns.shape[0])
    ]
    even_level_2_tuples = list(zip(*even_level_2_arrays))
    fcst_error_df.columns = pd.MultiIndex.from_tuples(even_level_2_tuples, names=["feature", "ufi", "hours_fwd"])

    return pd.concat([fcst_error_df, pivot_df], axis=1)


def get_master(date_from, date_to, cols_to_keep, horizons, ufis, values_to_pivot, do_pivot=True):

    # Load premaster data
    config_dict = Config.get_config()
    factory = DataAccessFactory()
    data_config = config_dict["data_access_factory"]
    source = factory.get(data_config["master_overcost"]["source"])

    master = utils_io.load_monthly(
        path=f"forecast/research/premaster_eolic",
        date_col="date",
        date_from=date_from,
        date_to=date_to,
        data_access=source,
    )

    # Get sample of premaster
    if cols_to_keep == "all":
        cols_to_keep = master.columns
    reduced_df = master[cols_to_keep]
    # Get only info for the next three hours
    reduced_df = reduced_df[reduced_df["hours_fwd"].isin(horizons)]
    # Get only records for target ufis
    reduced_df = reduced_df[reduced_df["ufi"].isin(ufis)][cols_to_keep]
    # Drop columns with empty meterings
    reduced_df = reduced_df[reduced_df['metering'].notna()]
    # Add forecast_error_predict_time
#     reduced_df["forecast_error_metering"] = reduced_df["forecast"] - reduced_df["metering"]

    # The telemetry is not aligned with the forecast thus we cannot simply subtract
    #     reduced_df["forecast_error_telemetry"] = reduced_df["forecast"] - reduced_df["telemetry"]

    # ?Drop rows with empty forecast error since we cannot know their real values 
    reduced_df = reduced_df.drop_duplicates()
    if do_pivot:
        pivot_df = pivot_master_by_levels(reduced_df)
    else:
        return reduced_df

    # Now we can align the forecasts and telemetry at market time to get the recent forecast error
    pivot_df = add_forecast_error_pivot(pivot_df, error_reference="telemetry")
    pivot_df = add_forecast_error_pivot(pivot_df, error_reference="metering")

    # It's really important to determine the order of the columns since we will be working with their array representation, not the dataframe
    pivot_df = pivot_df.sort_index(axis='columns', level=[0,1,2])

    return pivot_df



#### Split Sequences (time rows)

In [50]:
def split_sequences(sequences, n_steps_in, n_steps_out, target_col, portfolio_level=True):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences.iloc[i:end_ix, :], sequences.iloc[end_ix:out_end_ix, :]

        # As a feature we have forecast error based on telemetry (real time proxy for meterings)
        seq_x = seq_x.rename(columns={"forecast_error_telemetry":"forecast_error"})
        # As a target we want the forecast error to be the based on the actual metering
        seq_y = seq_y.rename(columns={"forecast_error_metering":"forecast_error"})

        # Drop the spare forecast error and original meterings
        seq_x = seq_x.drop(["forecast_error_metering","metering"], axis=1)

        # OPTION 1. Predict all input time series
        # seq_y = seq_y.drop(["forecast_error_telemetry","metering"], axis=1)
        # OPTION 2. Predict only target col for each UFI
        seq_y = seq_y[target_col]
        # OPTION 3. Predict only taget col for complete Portfolio
        if portfolio_level:
            if target_col == "forecast":
                seq_y = seq_y.loc[:, (slice(None),1)]
            seq_y = seq_y.sum(axis=1)

        # Flatten col levels
        flatten_cols_x = ["-".join(str(cs) for cs in c) for c in seq_x.columns.to_series()]
        seq_x.columns = flatten_cols_x
        if not portfolio_level:
            flatten_cols_y = ["-".join(str(cs) for cs in c) for c in seq_y.columns.to_series()]
            seq_y.columns = flatten_cols_y

        X.append(seq_x.values)
        y.append(seq_y.values)

    X_arr = array(X)
    y_arr = array(y)

    if portfolio_level:
        y_arr = y_arr[:,:, np.newaxis]

    return X_arr, y_arr

## Scene Generation


First we need a dataframe with the locations of each ufi.
Since we have some ufi sharing location (probably because the coords are specified at up level) we need to slightly offset them so that the algorithm can differentiate them

In [45]:
# date_first = "2018-12-31"
# date_last = "2022-10-31"
date_first = "2018-12-31"
date_last = "2019-12-31"
master = get_master(date_first, date_last, cols_to_keep="all", horizons=target_hours_fwd, ufis=target_ufis, values_to_pivot=info_columns)
master = master.sort_index()

2023-01-27 11:05:36,472 - MainThread - [INFO] - b'[UTILS IO] - load_monthly at line 176: Loading monthly from : forecast/research/premaster_eolic'
2023-01-27 11:05:36,507 - MainThread - [INFO] - b'[UTILS IO] - format_dates at line 465: 2018-12-31'
2023-01-27 11:05:36,652 - MainThread - [INFO] - b"[UTILS IO] - filter_files_load_monthly at line 448: Files in path: ['premaster_eolic___202209_0.h5', 'premaster_eolic___202210_0.h5', 'premaster_eolic___202009_0.h5', 'premaster_eolic___202101_0.h5', 'premaster_eolic___202008_0.h5', 'premaster_eolic___201912_0.h5', 'premaster_eolic___202104_0.h5', 'premaster_eolic___201905_0.h5', 'premaster_eolic___202102_0.h5', 'premaster_eolic___202205_0.h5', 'premaster_eolic___202207_0.h5', 'premaster_eolic___202012_0.h5', 'premaster_eolic___202005_0.h5', 'premaster_eolic___202002_0.h5', 'premaster_eolic___202006_0.h5', 'premaster_eolic___201904_0.h5', 'premaster_eolic___202208_0.h5', 'premaster_eolic___201906_0.h5', 'premaster_eolic___202001_0.h5', 'premas

In [5]:
# master_bl = master.copy()

In [6]:
# master = master_bl.copy()

In [7]:
# Get ufi list
ufi_list = master.columns.get_level_values(1)
# Filter out empty ufi used for  index levels and remove duplicates
ufi_list = list(filter(None, ufi_list.unique()))

ufi_coord_df = pd.DataFrame()
for ufi in ufi_list:
    # Use the tail since the head has some nans
    lat_ufi = master[~np.isnan(master["lat"][ufi])]["lat"][ufi].values[0]
    lon_ufi = master[~np.isnan(master["lon"][ufi])]["lon"][ufi].values[0]

    ufi_info = pd.Series([ufi,lat_ufi,lon_ufi])
    ufi_coord_df = ufi_coord_df.append(ufi_info, ignore_index=True)

ufi_coord_df.columns = ["ufi", "lat", "lon"]

repeated_coords = ufi_coord_df.groupby(["lat", "lon"]).agg(set).reset_index()
for lat,lon,ufis in repeated_coords[repeated_coords["ufi"].apply(lambda x: len(x)) > 1].values:

    offset = 1e-5
    for ufi in list(ufis):
        print(f"Offsetting {ufi}...")
        ufi_coord_df.loc[ufi_coord_df["ufi"] == ufi,"lat"] = lat + offset
        offset = offset + 1e-5
ufi_coord_df = ufi_coord_df.sort_values("ufi").reset_index(drop=True).reset_index() 
ufi_coord_df = ufi_coord_df.rename(columns={"index": "id"})

Offsetting HINOJII...
Offsetting HINOJAI...
Offsetting ZARZUEL...
Offsetting DEFERII...
Offsetting MUDEFER...
Offsetting FEIXOS...
Offsetting POTRA...
Offsetting BANDELE...
Offsetting RODERA...
Offsetting PESLOB...
Offsetting PESLOD...
Offsetting PESLOC...
Offsetting PESLOA...
Offsetting BAYO...
Offsetting ELLLAN...
Offsetting DEHESII...
Offsetting DEHEII...
Offsetting MONTOUT...
Offsetting PEARBO...
Offsetting ZAPATER...
Offsetting ELGALLO...
Offsetting PEZARZU...
Offsetting FRAILA...
Offsetting LACAYA...
Offsetting LAMESA...
Offsetting OTERO...
Offsetting PEOUROL...
Offsetting PECOUTE...
Offsetting SABUCED...
Offsetting VILACHA...


Check that there are no more duplicated coords

In [8]:
ufi_coord_df.groupby(["lat","lon"]).filter(lambda x: x.count()["ufi"]>1)

,id,ufi,lat,lon


### Grid embedding


In [9]:
sorted_latitudes = ufi_coord_df["lat"].unique()
sorted_latitudes.sort()

sorted_longitudes = ufi_coord_df["lon"].unique()
sorted_longitudes.sort()

grid_shape = [len(sorted_latitudes), len(sorted_longitudes)]
grid = np.ones(grid_shape)
grid = grid * -1


for id, ufi, lat, lon in ufi_coord_df.values:
    id_x = np.where(sorted_latitudes == lat)[0][0]
    id_y = np.where(sorted_longitudes == lon)[0][0]
    grid[id_x][id_y] = id





In [10]:
np.unique(grid)

array([-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
       12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.,
       25., 26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37.,
       38., 39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50.,
       51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63.])

In [11]:
grid.shape

(64, 45)

## Picture Master Generation

In [12]:
# # Flatten col levels
# master_og_cols = master.columns
# flatten_cols_x = ["-".join(str(cs) for cs in c) for c in master.columns.to_series()]
# master.columns = flatten_cols_x

In [13]:

def generate_scenes_master(master_pivot, grid_coords, localized_ufis):
    """
    :param master_pivot: data to train in pivotted format (type: pd.DataFrame)
    :param grid_coords: grid of size unique_latitutes X unique longitudes
                        with the id of the ufi located in each index
                        (type: np.array)
    :param localized_ufis: contains corrdinate information about each ufi
                           in master_pivot: id - ufi_name - lat - lon
                           (type: pd.DataFrame)

    : returns scene_master: Size: [time steps] X [latitudes] X [longitudes] X [features]
              features_index: data to relable the features. Size: [features]
    """

    example_ufi = master_pivot.columns.get_level_values(1)[-1]
    features_index = master_pivot.iloc[0, master_pivot.columns.get_level_values(1) == example_ufi].index
    
    num_features = len(features_index)
    num_time_steps = master_pivot.shape[0]

    # scene_master = [time steps] X [latitudes] X [longitudes] X [features]
    # same as image = [frame] X [width] X [length] X [channels]
    scene_master = np.ones((num_time_steps, grid_coords.shape[0], grid_coords.shape[1], num_features))
    scene_master = scene_master * -1


    # Iterate over the whole grid
    for lat_idx in np.arange(0,grid_coords.shape[0]):
        for lon_idx in np.arange(0,grid_coords.shape[1]):
            # Check the id for the position
            id = grid_coords[lat_idx][lon_idx]
            if id != -1:
                # Lookup its corresponding ufi
                ufi = localized_ufis[localized_ufis["id"] == id]["ufi"].values[0]
                # Retrieve data for ufi at time t
                features_for_ufi_all_t = master_pivot.iloc[:, master_pivot.columns.get_level_values(1) == ufi].values

                scene_master[:, lat_idx, lon_idx, :] = features_for_ufi_all_t

    return scene_master, features_index



In [14]:
master_picture, features_index_picture = generate_scenes_master(master, grid, ufi_coord_df)

In [15]:
master_picture.shape

(8758, 64, 45, 21)

In [16]:
features_index_picture.shape

(21,)

#### Check some values

In [17]:
ufi_to_check = "PESLOD"
t_to_check = 2
###################
id_to_check = ufi_coord_df[ufi_coord_df["ufi"] ==ufi_to_check]["id"].values
coords_to_check = np.where(grid == id_to_check)
lat_to_check = coords_to_check[0][0]
lon_to_check = coords_to_check[1][0]

In [18]:
ufi_coord_df[ufi_coord_df["ufi"] ==ufi_to_check]

,id,ufi,lat,lon
49,49,PESLOD,41.70902,-7.98


In [19]:
np.where(grid == id_to_check)

(array([22]), array([11]))

In [20]:
master_picture[t_to_check,lat_to_check,lon_to_check,0]

5.699999809265137

In [21]:
# Above value is 0 index which corresponds to forecast_1h_fwd
master.iloc[t_to_check, master.columns.get_level_values(1) == ufi_to_check]

feature                   ufi     hours_fwd
forecast                  PESLOD  1.0            5.700000
                                  2.0            4.400000
                                  3.0            4.000000
forecast_error_metering   PESLOD                 4.781000
forecast_error_telemetry  PESLOD                 4.405032
lat                       PESLOD                41.709000
lon                       PESLOD                -7.980000
metering                  PESLOD  1.0            0.919000
                                  2.0            1.531000
                                  3.0            1.728000
p_max                     PESLOD                33.700001
p_min                     PESLOD                      NaN
telemetry                 PESLOD                 2.394968
telemetry_close           PESLOD                 2.636870
telemetry_max             PESLOD                 3.017930
telemetry_min             PESLOD                 1.775700
telemetry_open            PE

In [22]:
features_index_picture

MultiIndex([(                'forecast', 'ZARZUEL', 1.0),
            (                'forecast', 'ZARZUEL', 2.0),
            (                'forecast', 'ZARZUEL', 3.0),
            ( 'forecast_error_metering', 'ZARZUEL',  ''),
            ('forecast_error_telemetry', 'ZARZUEL',  ''),
            (                     'lat', 'ZARZUEL',  ''),
            (                     'lon', 'ZARZUEL',  ''),
            (                'metering', 'ZARZUEL', 1.0),
            (                'metering', 'ZARZUEL', 2.0),
            (                'metering', 'ZARZUEL', 3.0),
            (                   'p_max', 'ZARZUEL',  ''),
            (                   'p_min', 'ZARZUEL',  ''),
            (               'telemetry', 'ZARZUEL',  ''),
            (         'telemetry_close', 'ZARZUEL',  ''),
            (           'telemetry_max', 'ZARZUEL',  ''),
            (           'telemetry_min', 'ZARZUEL',  ''),
            (          'telemetry_open', 'ZARZUEL',  ''),
            ( 

### Free up some memory

In [23]:
master = []
grid = []
ufi_coord_df = []

### Generate observations

In [49]:
def split_sequences_scenes(sequences, n_steps_in, n_steps_out, target_col, features_index, portfolio_level=True):
    X, y = list(), list()
    # Get indexes for the features to be used in each list
    flattened_cols = ["-".join(str(c[i]) for i in [0,2]) for c in features_index.to_series()]
    features_idx = [flattened_cols.index(col) for col in flattened_cols if ("metering" not in col) & ("lat" not in col) & ("lon" not in col)]
    features_idy = flattened_cols.index(target_col)

    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break

        # Divide the master rows for the current observation: observed vs predicted
        # Keep the features in seq_x and the target in seq_y
        seq_x = master_picture[i:end_ix, :,:,features_idx].copy()
        seq_y = master_picture[end_ix:out_end_ix, :,:,features_idy].copy()

        # (NOT TESTED FOR Spatial model) OPTION 1. Predict all input time series
        # seq_y = seq_y.drop(["forecast_error_telemetry","metering"], axis=1)

        # OPTION 3. Predict only taget col for complete Portfolio
        if portfolio_level:
            # Mask nans as -1
            seq_y = np.nan_to_num(seq_y, nan=-1)
            # Substitue values of non existing ufis (marked as -1) by a 0
            np.place(seq_y, seq_y==-1, 0)
            # Sum all target values to get the portfolio value for each timestep
            seq_y = seq_y.sum(axis=1).sum(axis=1)

        X.append(seq_x)
        y.append(seq_y)

    X_arr = array(X)
    y_arr = array(y)

    if portfolio_level:
        y_arr = y_arr[:,:, np.newaxis]

    return X_arr, y_arr

In [25]:
X_train, y_train = split_sequences_scenes(master_picture, n_steps_in, n_steps_in, target_col="forecast_error_metering-", features_index=features_index_picture)

In [26]:
X_train.shape

(8735, 12, 64, 45, 15)

In [27]:
y_train.shape

(8735, 12, 1)

#### Check some values

In [28]:
ufi_to_check = "PESLOD"
t_to_check = 0
###################
id_to_check = ufi_coord_df[ufi_coord_df["ufi"] == ufi_to_check]["id"].values
coords_to_check = np.where(grid == id_to_check)
lat_to_check = coords_to_check[0][0]
lon_to_check = coords_to_check[1][0]

TypeError: list indices must be integers or slices, not str

In [ ]:
ufi_coord_df[ufi_coord_df["ufi"] ==ufi_to_check]

In [ ]:
np.where(grid == id_to_check)

In [ ]:
master_picture[t_to_check,lat_to_check,lon_to_check,0]

In [ ]:
# Above value is 0 index which corresponds to forecast_1h_fwd
master.iloc[(t_to_check):(t_to_check + n_steps_in), master.columns.get_level_values(1) == ufi_to_check]

In [31]:
X_train.shape

(8735, 12, 64, 45, 15)

In [32]:
X_train[t_to_check,:,lat_to_check, lon_to_check,0]

array([6.80000019, 6.9000001 , 5.69999981, 4.19999981, 4.        ,
       4.5       , 4.9000001 , 4.19999981, 2.79999995, 1.10000002,
       0.60000002, 0.1       ])

In [33]:
y_train.shape

(8735, 12, 1)

In [34]:
y_train[t_to_check,:]

array([[12.8450007 ],
       [10.26700064],
       [15.34799976],
       [16.55999917],
       [ 7.35299991],
       [19.21099924],
       [11.78300006],
       [12.90299525],
       [43.58099956],
       [74.79400639],
       [49.3060046 ],
       [39.08300024]])

In [35]:
master.iloc[(n_steps_in + t_to_check):(n_steps_in + n_steps_out + t_to_check)]["forecast_error_metering"].fillna(0).sum(axis=1)

2019-01-01 13:00:00    12.845001
2019-01-01 14:00:00    10.267000
2019-01-01 15:00:00    15.348000
dtype: float32

## Train Model & Predict

In [30]:
from tensorflow import keras 
import tensorflow as tf
from keras import layers
from keras import Input, Model, metrics
from keras.layers import Dense, RepeatVector, LSTM, TimeDistributed, Dropout
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [33]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(n_steps_in,*X_train.shape[2:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
# x = RepeatVector(n_steps_out)(inp)
# x = layers.ConvLSTM2D(
#     filters=21,
#     kernel_size=(5, 5),
#     padding="same",
#     return_sequences=True,
#     activation="relu",
# )(inp)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same')(x)

x = layers.ConvLSTM2D(
    filters=10,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling3D(pool_size=(1, 3, 3), padding='same')(x)

x = layers.ConvLSTM2D(
    filters=3,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same')(x)
# x = layers.Conv3D(
#     filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
# )(x)
# x = layers.Conv2D(1, (1,1), activation='softmax')(x)
# x = layers.AveragePooling3D()(x)
flatten = tf.keras.layers.Flatten()
x = TimeDistributed(flatten)(x)
# outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten1)
dense_1 = Dense(256,activation="relu")
dense_2 = Dense(32,activation="relu")
dense_3 = Dense(1,activation="relu")
x = TimeDistributed(dense_1)(x)
x = TimeDistributed(dense_2)(x)
x = TimeDistributed(dense_3)(x)


# input_layer = Input(shape=(self.n_steps_in,self.n_features_in), name='input_layer')
# lstm_1 = LSTM(30, activation='relu', name="LSTM_Layer_1")(input_layer)
# repeat_vector = RepeatVector(self.n_steps_out, name="Repeating_Vector_Layer")(lstm_1)
# lstm_2 = LSTM(30, activation='relu', return_sequences=True, name="LSTM_Layer_2")(repeat_vector)
# lstm_3 = LSTM(10, activation='relu', return_sequences=True, name="LSTM_Layer_3")(lstm_2)
# dense = Dense(self.n_features_out, activation="relu",name='Dense_Layer')
# dropout = Dropout(.3, input_shape=(2,))(lstm_3)
# time_dist = TimeDistributed(dense, name='Time_Distributed_Layer')(dropout)

In [34]:
# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.compile(
    loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(),
)

In [35]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 12, 64, 45, 15)]  0         
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 12, 64, 45, 10)    9040      
                                                                 
 batch_normalization_1 (Batc  (None, 12, 64, 45, 10)   40        
 hNormalization)                                                 
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 12, 22, 15, 10)   0         
 3D)                                                             
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, 12, 22, 15, 3)     168       
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 12, 11, 8, 3)     0     

In [37]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
# reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 20
batch_size = 100

# Fit the model to the training data.
size_train = np.floor(X_train.shape[0] * 0.7)
size_validate = np.floor(X_train.shape[0] * 0.3)+1

X_val = X_train[int(size_train):]
y_val = y_train[int(size_train):]

model.fit(
    np.nan_to_num(X_train[:int(size_train)], nan=-1),
    np.nan_to_num(y_train[:int(size_train)], nan=-1),
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(np.nan_to_num(X_val, nan=-1), np.nan_to_num(y_val, nan=-1)),
    callbacks=[early_stopping],# reduce_lr],
    verbose=1
)

Epoch 1/20
62/62 [==============================] - 522s 8s/step - loss: -45.7694 - val_loss: -116.7509
Epoch 2/20
62/62 [==============================] - 505s 8s/step - loss: -63.3853 - val_loss: -145.5883
Epoch 3/20
62/62 [==============================] - 506s 8s/step - loss: -66.1240 - val_loss: -145.5883
Epoch 4/20
62/62 [==============================] - 504s 8s/step - loss: -66.1240 - val_loss: -145.5883
Epoch 5/20
62/62 [==============================] - 505s 8s/step - loss: -66.1240 - val_loss: -145.5883
Epoch 6/20
62/62 [==============================] - 504s 8s/step - loss: -66.1240 - val_loss: -145.5883
Epoch 7/20
62/62 [==============================] - 504s 8s/step - loss: -66.1240 - val_loss: -145.5883


In [39]:
model.save('data/tfm/convlstm_model.h5')

In [85]:
y_train.shape

(8744, 3, 1)

In [ ]:
n_steps_out

In [102]:
y_train[:int(size_train)].shape

(6120, 3, 1)

In [42]:
predictions = model.predict(np.nan_to_num(X_val, nan=-1))

82/82 [==============================] - 27s 329ms/step


## Performance

In [47]:
predictions.shape

(2621, 12, 1)

In [48]:
y_val.shape

(2621, 12, 1)

In [53]:
y_train.shape

(8735, 12, 1)

In [54]:
master.shape

(8758, 1347)

In [65]:
target_col = "forecast_error_metering-"

In [179]:
_, y_forecast = split_sequences(master.iloc[int(size_train):],n_steps_in,n_steps_in,target_col="forecast")


In [171]:
master_test_format = master.iloc[int(size_train):].rename(columns={"forecast_error_metering":"forecast_error"}).copy()

In [172]:
master_test_format["forecast_error_portfolio"] = master_test_format["forecast_error"].sum(axis=1)

In [173]:
prediction_col_schema = ["forecast_error_portfolio"]

In [174]:
forecast_col_schema = ["forecast_portfolio"]

In [175]:
horizons = np.arange(0, n_steps_out)

In [176]:
horizons

array([0, 1, 2])

In [177]:
size_train

6114.0

In [180]:
master_test_format["forecast"][ufi_to_check].head(20)

hours_fwd,1.0,2.0,3.0
2019-09-12 19:00:00,13.900000,18.799999,19.900000
2019-09-12 20:00:00,18.799999,19.900000,19.000000
2019-09-12 21:00:00,19.900000,19.000000,17.799999
2019-09-12 22:00:00,19.000000,17.799999,16.700001
2019-09-12 23:00:00,17.799999,16.700001,14.700000
2019-09-13 00:00:00,16.700001,14.700000,13.500000
2019-09-13 01:00:00,14.500000,13.500000,14.300000
2019-09-13 02:00:00,13.800000,14.300000,15.600000
2019-09-13 03:00:00,14.300000,15.600000,16.799999
2019-09-13 04:00:00,14.900000,16.799999,16.400000


In [190]:
X_val[t_to_check,:,lat_to_check,lon_to_check,0]

array([13.89999962, 18.79999924, 19.89999962, 19.        , 17.79999924,
       16.70000076, 14.5       , 13.80000019, 14.30000019, 14.89999962,
       15.89999962, 16.39999962])

In [191]:
master_test_format["forecast_error_portfolio"].head(20)

2019-09-12 19:00:00    118.731003
2019-09-12 20:00:00     72.023994
2019-09-12 21:00:00    149.980011
2019-09-12 22:00:00    185.714005
2019-09-12 23:00:00    126.595993
2019-09-13 00:00:00     86.719002
2019-09-13 01:00:00     83.042007
2019-09-13 02:00:00     74.493004
2019-09-13 03:00:00    107.165001
2019-09-13 04:00:00    114.043991
2019-09-13 05:00:00     39.010002
2019-09-13 06:00:00     36.958000
2019-09-13 07:00:00     68.350006
2019-09-13 08:00:00     67.881004
2019-09-13 09:00:00     96.738998
2019-09-13 10:00:00     56.345005
2019-09-13 11:00:00     19.947001
2019-09-13 12:00:00     50.047009
2019-09-13 13:00:00      1.284002
2019-09-13 14:00:00    -48.785999
Name: forecast_error_portfolio, dtype: float32

In [192]:
y_val[t_to_check,:,:]
# master_picture[t_to_check,lat_to_check,lon_to_check,0]

array([[ 68.35000446],
       [ 67.88100625],
       [ 96.73899926],
       [ 56.34500118],
       [ 19.9469992 ],
       [ 50.04700507],
       [  1.2840028 ],
       [-48.78599495],
       [-65.74800631],
       [-61.61499258],
       [ -2.97500289],
       [-22.35500249]])

In [193]:
pd.DataFrame(y_val[:,t_plus,:], columns=prediction_col_schema)

,forecast_error_portfolio
0,68.350004
1,67.881006
2,96.738999
3,56.345001
4,19.946999
...,...
2616,10.303001
2617,4.393000
2618,2.808001
2619,-6.562000


In [194]:
t_plus = 0
head_offset = n_steps_in+t_plus
tail_offset = -n_steps_in+t_plus+1

In [195]:
master_test_format.index[head_offset:tail_offset]

DatetimeIndex(['2019-09-13 07:00:00', '2019-09-13 08:00:00',
               '2019-09-13 09:00:00', '2019-09-13 10:00:00',
               '2019-09-13 11:00:00', '2019-09-13 12:00:00',
               '2019-09-13 13:00:00', '2019-09-13 14:00:00',
               '2019-09-13 15:00:00', '2019-09-13 16:00:00',
               ...
               '2019-12-31 02:00:00', '2019-12-31 03:00:00',
               '2019-12-31 04:00:00', '2019-12-31 05:00:00',
               '2019-12-31 06:00:00', '2019-12-31 07:00:00',
               '2019-12-31 08:00:00', '2019-12-31 09:00:00',
               '2019-12-31 10:00:00', '2019-12-31 11:00:00'],
              dtype='datetime64[ns]', length=2621, freq=None)

In [196]:
# Predictions for forecast error
predictions_t = pd.DataFrame(predictions[:,t_plus,:], columns=prediction_col_schema)
predictions_t.index = master_test_format.index[head_offset:tail_offset]

In [197]:
test_t = pd.DataFrame(y_val[:,t_plus,:], columns=prediction_col_schema)
test_t.index = master_test_format.index[head_offset:tail_offset]

In [199]:
# Forecast
forecast_t = pd.DataFrame(y_forecast[:,t_plus,:], columns=forecast_col_schema)
forecast_t.index = master_test_format.index[head_offset:tail_offset]

In [251]:
imbalance_row = ["portfolio"]
accuracy_row_mae = ["portfolio"]
cols_to_report = ["ufi"] + ["t_1"]

imbalance_report = pd.DataFrame()
accuracy_report_mae = pd.DataFrame()
rmse = RootMeanSquaredError()
mae = MeanAbsoluteError()

In [252]:
t = 0
prediction_forecast_error_metering = predictions_t.iloc[:,t].values
test_set_forecast_error_metering = test_t.iloc[:,t].values
baseline_forecast = forecast_t.iloc[:,t].values
non_null_indices = np.argwhere(~np.isnan(baseline_forecast))

In [253]:
baseline_forecast = baseline_forecast[non_null_indices]
prediction_forecast_error_metering = prediction_forecast_error_metering[non_null_indices]
test_set_forecast_error_metering = test_set_forecast_error_metering[non_null_indices]

In [254]:
expected_forecast = baseline_forecast + prediction_forecast_error_metering
true_forecast = baseline_forecast + test_set_forecast_error_metering  # Should be equal to metering

In [255]:
baseline_imbalance = np.sum(true_forecast) - np.sum(baseline_forecast)  # Should be equal to forecast_error_metering
expected_imbalance = np.sum(true_forecast) - np.sum(expected_forecast)
delta_imbalance = np.abs(baseline_imbalance) - np.abs(expected_imbalance)

In [256]:
mae_ufi_t = mae(test_set_forecast_error_metering,prediction_forecast_error_metering).numpy()

In [257]:
imbalance_row = imbalance_row + [delta_imbalance]
accuracy_row_mae = accuracy_row_mae + [mae_ufi_t]

In [258]:
imbalance_report = imbalance_report.append(pd.Series(imbalance_row, index=cols_to_report), ignore_index=True)
accuracy_report_mae = accuracy_report_mae.append(pd.Series(accuracy_row_mae, index=cols_to_report), ignore_index=True)

In [259]:
imbalance_report

,t_1,ufi
0,5307.0,portfolio


In [260]:
accuracy_report_mae

,t_1,ufi
0,44.254787,portfolio


In [43]:
def format_outputs(master_test, predictions, n_steps_in, n_steps_out, portfolio_level=True):

    print("Get forecasts from master test")
    _, y_forecast = split_sequences(master_test,n_steps_in,n_steps_out,target_col="forecast")
    # Get col schema for raw predictions
    master_test_format = master_test.rename(columns={"forecast_error_metering":"forecast_error"}).copy()

    if portfolio_level:
        master_test_format["forecast_error_portfolio"] = master_test_format["forecast_error"].sum(axis=1)
        prediction_col_schema = ["forecast_error_portfolio"]
        forecast_col_schema = ["forecast_portfolio"]
    else:
        prediction_col_schema = master_test_format["forecast_error"].columns
        # Get col schema for forecasts to compute imbalances
        forecasts_formatted = master_test.loc[:, (slice("forecast"), slice(None), slice(None))]
        forecast_col_schema = forecasts_formatted["forecast"].columns

    horizons = np.arange(0, n_steps_out)

    predictions_formatted = pd.DataFrame()
    test_set_formatted = pd.DataFrame()
    forecast_set_formatted = pd.DataFrame()
    predictions_t = pd.DataFrame()
    print("Formatting predictions")
    for t_plus in horizons:
        head_offset = n_steps_in+t_plus
        tail_offset = -n_steps_out+t_plus+1
        if tail_offset == 0:
            tail_offset = None

        # Predictions for forecast error
        predictions_t = pd.DataFrame(predictions[:,t_plus,:], columns=prediction_col_schema)
        predictions_t.index = master_test_format.index[head_offset:tail_offset]

        if t_plus == horizons[0]:
            predictions_formatted = predictions_t
        else:
            predictions_formatted = predictions_formatted.join(predictions_t, rsuffix=f"_t{t_plus+1}")

        # True forecast error
        test_t = pd.DataFrame(y_test[:,t_plus,:], columns=prediction_col_schema)
        test_t.index = master_test_format.index[head_offset:tail_offset]

        if t_plus == horizons[0]:
            test_set_formatted = test_t
        else:
            test_set_formatted = test_set_formatted.join(test_t, rsuffix=f"_t{t_plus+1}")

        # Forecast
        forecast_t = pd.DataFrame(y_forecast[:,t_plus,:], columns=forecast_col_schema)
        forecast_t.index = master_test_format.index[head_offset:tail_offset]

        if t_plus == horizons[0]:
            forecast_set_formatted = forecast_t
        else:
            forecast_set_formatted = forecast_set_formatted.join(forecast_t, rsuffix=f"_t{t_plus+1}")

    return predictions_formatted, test_set_formatted, forecast_set_formatted

In [204]:
def compute_accuracy(predictions_formatted, test_set_formatted, forecast_set_formatted, n_steps_out):
   
    horizons = np.arange(0, n_steps_out)
    cols_to_report = ["ufi"] + list(horizons)

    imbalance_row = ["portfolio"]
    accuracy_row_mae = ["portfolio"]

    imbalance_report = pd.DataFrame()
    accuracy_report_mae = pd.DataFrame()
    rmse = RootMeanSquaredError()
    mae = MeanAbsoluteError()

    for t in horizons:

        if t == 0:
            suffix = ""
        else:
            suffix = f"_t{t+1}"

        prediction_forecast_error_metering = predictions_formatted.iloc[:,t].values
        test_set_forecast_error_metering = test_set_formatted.iloc[:,t].values

        # Due to the way we query the forecasts there are gaps which result in nan values
        baseline_forecast = forecast_set_formatted.iloc[:,t].values
        non_null_indices = np.argwhere(~np.isnan(baseline_forecast))
        # Filter out data without target info
        baseline_forecast = baseline_forecast[non_null_indices]
        prediction_forecast_error_metering = prediction_forecast_error_metering[non_null_indices]
        test_set_forecast_error_metering = test_set_forecast_error_metering[non_null_indices]

        expected_forecast = baseline_forecast + prediction_forecast_error_metering
        true_forecast = baseline_forecast + test_set_forecast_error_metering  # Should be equal to metering

        baseline_imbalance = np.sum(true_forecast) - np.sum(baseline_forecast)  # Should be equal to forecast_error_metering
        expected_imbalance = np.sum(true_forecast) - np.sum(expected_forecast)
        delta_imbalance = np.abs(baseline_imbalance) - np.abs(expected_imbalance)

        mae_ufi_t = mae(test_set_forecast_error_metering,prediction_forecast_error_metering).numpy()
    
        imbalance_row = imbalance_row + [delta_imbalance]
        accuracy_row_mae = accuracy_row_mae + [mae_ufi_t]

    imbalance_report = imbalance_report.append(pd.Series(imbalance_row, index=cols_to_report), ignore_index=True)
    accuracy_report_mae = accuracy_report_mae.append(pd.Series(accuracy_row_mae, index=cols_to_report), ignore_index=True)
    
    return imbalance_report, accuracy_report_mae